# Singapore HDB Price Analysis (2024) (Using SQL)

In [120]:
import pandas as pd
import sqlite3

In [121]:
df = pd.read_csv('HDB_faltData.csv')
df['month'] = pd.to_datetime(df['month'])
df.rename(columns={
    'month': 'date'
}, inplace=True)
df = df[(df['date'] >= '2024-01-01') & (df['date'] < '2025-01-01')]

In [122]:
conn = sqlite3.connect('flatData.db')
df.to_sql('flat_data', conn, if_exists='replace', index=False)

27852

In [123]:
query = 'select * from flat_data limit 5'
pd.read_sql(query, conn)

,date,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2024-02-01 00:00:00,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,01 TO 03,44.0,Improved,1979,54 years 04 months,285000.0
1,2024-12-01 00:00:00,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,04 TO 06,44.0,Improved,1979,53 years 07 months,320000.0
2,2024-02-01 00:00:00,ANG MO KIO,2 ROOM,323,ANG MO KIO AVE 3,04 TO 06,44.0,Improved,1977,52 years 05 months,293000.0
3,2024-02-01 00:00:00,ANG MO KIO,2 ROOM,314,ANG MO KIO AVE 3,01 TO 03,44.0,Improved,1978,52 years 11 months,303000.0
4,2024-04-01 00:00:00,ANG MO KIO,2 ROOM,314,ANG MO KIO AVE 3,01 TO 03,44.0,Improved,1978,52 years 10 months,288000.0


## Q1: Which top 5 town has the highest average resale price?

In [124]:
query = 'select town, round(avg(resale_price),2) as avg_resale_price from flat_data group by town order by avg_resale_price desc limit 5'
pd.read_sql(query, conn)

,town,avg_resale_price
0,BUKIT TIMAH,883392.77
1,BISHAN,830693.71
2,CENTRAL AREA,757515.75
3,BUKIT MERAH,724272.81
4,PASIR RIS,703587.85


Highest average resale price: **BUKIT TIMAH** (SGD 883k) 

## Q2: How do resale prices trend over time?

In [148]:
query = 'select strftime("%m", date) as month, round(avg(resale_price),2) as avg_flat_price from flat_data group by strftime("%m", date)'
trend = pd.read_sql(query, conn)

In [147]:
price_Jan = trend.iloc[0]['avg_flat_price']
price_Dec = trend.iloc[-2]['avg_flat_price']
price_increase_percent = round(((price_Dec - price_Jan) / price_Jan) * 100)
print(f'The price incresed by {price_increase_percent}%.')

The price incresed by 7%.


Prices increased by **7% YoY** from JAN to DEC during 2024.  

## Q3: Which flat type is most expensive?

In [127]:
query = 'select flat_type, round(avg(resale_price),2) as avg_flat_price from flat_data group by flat_type order by avg_flat_price desc'
pd.read_sql(query, conn)

,flat_type,avg_flat_price
0,MULTI-GENERATION,1049000.00
1,EXECUTIVE,874650.59
2,5 ROOM,728615.22
3,4 ROOM,627382.84
4,3 ROOM,439910.47
5,2 ROOM,340847.70
6,1 ROOM,253363.64


Most expensive flat type: **MULTI-GENERATION** (avg. SGD 1049k).

## Q4: Which towns have average resale prices higher than the national average?

In [152]:
query = '''
        select town, avg(resale_price) as avg_resale_price 
        from flat_data 
        group by town 
        having avg_resale_price > 
            (
                select avg(resale_price) from flat_data
            ) 
        order by avg_resale_price desc
        '''
pd.read_sql(query, conn)

,town,avg_resale_price
0,BUKIT TIMAH,883392.774194
1,BISHAN,830693.714660
2,CENTRAL AREA,757515.750000
3,BUKIT MERAH,724272.811538
4,PASIR RIS,703587.846266
5,KALLANG/WHAMPOA,691353.645276
6,QUEENSTOWN,685510.997139
7,SERANGOON,672018.977324
8,TOA PAYOH,664599.637376
9,TAMPINES,657439.314635


## Q5: Find flats where the price is 20% higher than the average price in their town.

In [151]:
query = '''
        select flat_type, d1.town, d1.resale_price 
        from 
                (
                        select flat_type, town, resale_price 
                        from flat_data 
                        group by flat_type, town
                ) as d1
                join 
                (
                        select town, avg(resale_price) as avg_price
                        from flat_data 
                        group by town
                ) as d2
                using (town)
        where resale_price > (120*avg_price)/100
        order by resale_price desc
        limit 5
        '''
pd.read_sql(query, conn)

,flat_type,town,resale_price
0,5 ROOM,CENTRAL AREA,1450000.0
1,EXECUTIVE,BISHAN,1380000.0
2,EXECUTIVE,BUKIT TIMAH,1360000.0
3,EXECUTIVE,ANG MO KIO,1238888.0
4,EXECUTIVE,QUEENSTOWN,1228000.0


## Q6: Compare each month’s average price to the previous month using subqueries.

In [153]:
query = '''
        SELECT
            strftime("%Y-%m", h1.month) AS month,
            round(h1.avg_price,2) as curr_month_avg,
            (
                SELECT round(AVG(resale_price),2)
                FROM flat_data
                WHERE strftime("%Y-%m", date) = strftime("%Y-%m", date(h1.month, '-1 month'))
            ) AS prev_month_avg
        FROM (
            SELECT
                date AS month,
                AVG(resale_price) AS avg_price
            FROM flat_data
            GROUP BY strftime("%Y-%m", date)
        ) h1;
        '''

pd.read_sql(query, conn)

,month,curr_month_avg,prev_month_avg
0,2024-01,591348.59,NaN
1,2024-02,585635.89,591348.59
2,2024-03,596945.80,585635.89
3,2024-04,592390.17,596945.80
4,2024-05,599861.58,592390.17
5,2024-06,620309.38,599861.58
6,2024-07,616816.87,620309.38
7,2024-08,621291.92,616816.87
8,2024-09,629234.95,621291.92
9,2024-10,629456.29,629234.95
